In [1]:
import pandas as pd
import requests
from lxml import etree
from time import sleep
import json

In [4]:
class Test_1_Spider():
    def __init__(self):
        #数据初始化
        self.basic_url = 'https://spa1.scrape.center/api/movie?limit=10&offset='
        self.headers = {
            "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
            "accept-encoding":"gzip, deflate, br, zstd",
            "accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "accept-language":"zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6",
            "connection":"keep-alive",
            "host":"spa1.scrape.center",
            "x-requested-with":"XMLHttpRequest"
        }
    
    #构造url，翻页
    def get_url_list(self):
            url_list = [self.basic_url+f"{i}" for i in range(0,100,10)]
            return url_list
    #获取响应
    def get_response(self,url):
        response_json = requests.get(url,headers=self.headers).json()
        return response_json
    #清洗数据
    def get_data(self,response_json):
        data = pd.DataFrame(columns=["电影名称","类型","发布时间","产地","时长","得分"])
        data_list = response_json.get("results")
        for d in data_list:
            info_dicts = {}
            info_dicts["电影名称"] = d.get("name")
            info_dicts["类型"] = "-".join(d.get("categories"))
            info_dicts["发布时间"] = d.get("published_at")
            info_dicts["产地"] = "-".join(d.get("regions"))
            info_dicts["时长"] = d.get("minute")
            info_dicts["得分"] = d.get("score")
            info_df = pd.DataFrame(info_dicts,index=[0])
            data = pd.concat([data,info_df])
        return data



    def run(self):
         #运行主程序
         #1、获取url列表
        need_data = pd.DataFrame(columns=["电影名称","类型","发布时间","产地","时长","得分"])
        url_list = self.get_url_list()
        for url in url_list:
            #2、发送请求，获取响应
            html_str = self.get_response(url)
            #3、清洗数据
            data = self.get_data(html_str)
            #保存数据
            need_data = pd.concat([need_data,data])
            sleep(2)
        need_data.to_excel(".\spider数据.xlsx",index=False)


In [5]:
if __name__ == "__main__":
    Spider = Test_1_Spider()
    Spider.run()